# Upload, register and deploy models from the 🤗 Hub

In order to deploy a model in Vertex AI, we will first need to upload it to a Google Cloud Storage (GCS) bucket, and then register it in Vertex AI; meaning that we cannot deploy straight from the HuggingFace Hub, but using GCS as the intermediate storage.

## Installation

* `gcloud` CLI needs to be installed and logged in the project that will be used. See the installation notes at https://cloud.google.com/sdk/docs/install

* `google-cloud-aiplatform` Python library is required to register the model and deploy it as an endpoint in Vertex AI.

    `pip install google-cloud-aiplatform --upgrade`

* `git lfs` needs to be installed for pulling / cloning models from the HuggingFace Hub. See the installation notes at https://git-lfs.com/.

## Setup

To successfully run the code below, you will need to be authenticated into your Google Cloud account and the following variable values must be set in advance:

* `REGION` is the region where the resources will be hosted in.
* `PROJECT_ID` is the identifier of the project in Google Cloud.
* `REPOSITORY` is the directory where the Docker images will be uploaded to.
* `IMAGE` is the name of the Docker image (without tag).
* `TAG` is that tag of the Docker image.
* `BUCKET_NAME` is the name of the bucket were the model will be / has been uploaded to.
* `BUCKET_URI` is the full path to the `model.tar.gz` file in Cloud Storage.

In [ ]:
REGION = "europe-west9"
PROJECT_ID = "huggingface-cloud"
REPOSITORY = "custom-inference"
IMAGE = "huggingface-pipeline"
TAG = "py310-cpu-torch-2.2.0-transformers-4.38.1"
SERVING_CONTAINER_IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}:{TAG}"
BUCKET_NAME = "huggingface-cloud"
BUCKET_URI = f"gs://{BUCKET_NAME}/bart-large-mnli/model.tar.gz"

## Upload model from the 🤗 Hub

First we need to decide which model from the HuggingFace Hub we want to use, in this case, we will be using `facebook/bart-large-mnli` which is a zero-shot classification model, but could be any model in the Hub.

In order to do so, we will pull the model from the HuggingFace Hub using `git pull`, which requires `git lfs` to be installed in advanced, in order to also pull the large files from the repository.

In [ ]:
!git lfs install
!git clone https://huggingface.co/facebook/bart-large-mnli

Once we clone it, we will need to decide which files we want to package into `model.tar.gz` and which models we want to leave outside i.e. the weights in other frameworks instead of the one we want to use e.g. `torch`, and any other file that may be part of the model repository that's not needed to load the model for inference.

In [ ]:
!cd bart-large-mnli/ && tar zcvf model.tar.gz --exclude flax_model.msgpack --exclude pytorch_model.bin --exclude rust_model.ot * && mv model.tar.gz ../

Once we've packaged all the required files into `model.tar.gz`, we can upload it to Google Cloud Storage, so that the URI pointing to that file will be later on provided to Vertex AI to register the model from that location.

In [ ]:
!gcloud config set storage/parallel_composite_upload_enabled True
!gcloud storage cp model.tar.gz $BUCKET_URI

Optionally, we can `gcloud storage ls` to ensure that the `model.tar.gz` file has been indeed uploaded to GCS.

In [ ]:
!gcloud storage ls --recursive gs://{BUCKET_NAME}

![](https://raw.githubusercontent.com/alvarobartt/vertex-ai-huggingface/main/online-prediction/assets/model-cloud-storage.png)

## Register model in Vertex AI

Once the model is uploaded to GCS, we can already register it in Vertex AI, but to do so we will also need to specify the container that will run that model in advance.

It could be any container with `python` and `pip` installed, and the required CPR requirements met, so that the endpoint can be deployed normally, otherwise the deployment will fail.

For more information check [`01-build-custom-inference-images.ipynb`](./01-build-custom-inference-images.ipynb).

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

In [ ]:
!gcloud auth login
!gcloud auth application-default login

So we will be using `google-cloud-aiplatform` to register a model from GCS into Vertex AI, which in this case will match the model previously uploaded to GCS.

_**Note**: that all the `serving_*` arguments of the classmethod `upload` from `aiplatform.Model` refer to the container that will be pulled from the container registry and used during inference when deploying the endpoint, which must have been created in advance, as explained in `01-build-custom-inference-images.ipynb`.

In [ ]:
model = aiplatform.Model.upload(
    display_name="bart-large-mnli",
    artifact_uri="gs://huggingface-cloud/bart-large-mnli",
    serving_container_image_uri=SERVING_CONTAINER_IMAGE_URI,
    serving_container_environment_variables={
        "HF_TASK": "zero-shot-classification",
        # Optional env var so that `uvicorn` only runs the model in 1 worker
        # "VERTEX_CPR_WEB_CONCURRENCY": 1,
    },
)

![](https://raw.githubusercontent.com/alvarobartt/vertex-ai-huggingface/main/online-prediction/assets/model-registry-vertex-ai.png)

## Deploy endpoint in Vertex AI

Finally, we can use the `aiplatform.Model` object returned before when the `upload` class method was called, to call the `deploy` method that will deploy an endpoint using FastAPI (unless the handler in the CPR was overwritten) running in a machine matching the `machine_type` argument.

In this case we will only define the `machine_type` arg, and we will use the `e2-standard-4` which is a VM from Compute Engine with 4 vCPUs and 16GiB of RAM.

To see all the possible args of `deploy` see the source code at [`google.cloud.aiplatform.models`](https://github.com/googleapis/python-aiplatform/blob/f249353b918823b35495b295a75a90528ad652c0/google/cloud/aiplatform/models.py#L3418).

_**Note**: the `deploy` method will take a while ~15-20 minutes in order to deploy the model in Vertex AI as an endpoint._

In [ ]:
endpoint = model.deploy(machine_type="e2-standard-4")

![](https://raw.githubusercontent.com/alvarobartt/vertex-ai-huggingface/main/online-prediction/assets/endpoint-deployment-vertex-ai.png)

## Online predictions in Vertex AI

Then, once the Vertex AI endpoint is running, we can try it out using the `PredictionServiceClient` from `google-cloud-aiplatform` to send an HTTP request to the `bart-base-mnli` model running the `zero-shot-classification` task.

In [ ]:
import json
from google.api import httpbody_pb2
from google.cloud import aiplatform_v1

prediction_client = aiplatform_v1.PredictionServiceClient(
    client_options={"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
)

data = {
    "sequences": "Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app.",
    "candidate_labels": ["mobile", "website", "billing", "account access"],
}

json_data = json.dumps(data)

http_body = httpbody_pb2.HttpBody(
    data=json_data.encode("utf-8"),
    content_type="application/json",
)

request = aiplatform_v1.RawPredictRequest(
    endpoint=endpoint.resource_name,
    http_body=http_body,
)

response = prediction_client.raw_predict(request)
json.loads(response.data)

## Resource clean up

Finally, we clean up the resources used i.e. the Vertex AI endpoint and the model. In this case, we clean the resources up because we are no longer going to use those, but alternatively we could leave those running.

In [ ]:
endpoint.delete(force=True)
model.delete()